# Importing Necessary Libraries

In [1]:
# Essential installations with evaluation tools
!pip install -q  --no-cache-dir transformers torch gradio accelerate safetensors pillow diffusers scikit-learn numpy pandas

import gc
import os
import torch
import warnings
import gradio as gr
from typing import Optional, List, Tuple, Any, Dict
from transformers.pipelines import pipeline
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
import time
from datetime import datetime
from PIL import Image
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 113.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 268.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 190.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 118.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 163.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 152.2 MB/s eta 0:00:00


# Model Pipeline Development

In [2]:
# Configure environment
warnings.filterwarnings("ignore")
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

class EvaluatedNLPTool:
    """
    NLP tool with comprehensive evaluation metrics based on project requirements
    """

    def __init__(self):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.current_pipeline = None
        self.current_task = None
        self.conversation_history = []
        self.evaluation_data = {}
        self.image_pipe = None  # Initialize image pipeline attribute
        self.task_feedback = {}  # Store feedback for each task

        # Model configurations
        self.model_configs = {
            "summarization": {
                "model_name": "facebook/bart-large-cnn",
                "task": "summarization"
            },
            "sentiment": {
                "model_name": "cardiffnlp/twitter-roberta-base-sentiment-latest",
                "task": "sentiment-analysis"
            },
            "qa": {
                "model_name": "deepset/roberta-base-squad2",
                "task": "question-answering"
            },
            "next_word": {
                "model_name": "gpt2-medium",
                "task": "text-generation"
            },
            "text_generation": {
                "model_name": "gpt2-large",  # Enhanced for story generation
                "task": "text-generation"
            },
            "chatbot": {
                "model_name": "gpt2-large",  # Upgraded to GPT-2 Large for better responses
                "task": "text-generation"
            }
        }

        print(f"🚀 EvaluatedNLPTool initialized on device: {self.device}")
        self._init_evaluation_storage()

    def _init_evaluation_storage(self):
        """Initialize evaluation metrics storage"""
        self.evaluation_data = {
            "accuracy": [],
            "precision": [],
            "recall": [],
            "f1_score": [],
            "user_satisfaction": [],
            "response_times": [],
            "task_performance": {},
            "task_feedback_history": []
        }

        # Initialize task-specific feedback storage
        self.task_feedback = {
            "summarization": {"ratings": [], "response_times": []},
            "sentiment": {"ratings": [], "response_times": []},
            "qa": {"ratings": [], "response_times": []},
            "next_word": {"ratings": [], "response_times": []},
            "story_generation": {"ratings": [], "response_times": []},
            "chatbot": {"ratings": [], "response_times": []},
            "image_generation": {"ratings": [], "response_times": []}
        }

    def clear_gpu_memory(self):
        """Enhanced GPU memory cleanup"""
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.synchronize()
        gc.collect()
        print("🧹 GPU memory cleared")

    def aggressive_memory_cleanup(self):
        """Aggressive memory cleanup for image generation"""
        if hasattr(self, 'image_pipe') and self.image_pipe is not None:
            del self.image_pipe
            self.image_pipe = None

        if self.current_pipeline is not None:
            del self.current_pipeline
            self.current_pipeline = None
            self.current_task = None

        self.clear_gpu_memory()
        time.sleep(2)  # Allow memory to be freed
        print("🚨 Aggressive memory cleanup completed")

    def load_pipeline_efficiently(self, task: str) -> bool:
        """Smart pipeline loading with enhanced memory management"""
        if self.current_task == task and self.current_pipeline:
            return True

        try:
            config = self.model_configs.get(task)
            if not config:
                return False

            if self.current_pipeline:
                del self.current_pipeline
                self.clear_gpu_memory()
                time.sleep(1)

            print(f"📥 Loading {task} model...")

            self.current_pipeline = pipeline(
                config["task"],
                model=config["model_name"],
                device=0 if self.device == "cuda" else -1,
                torch_dtype=torch.float16 if self.device == "cuda" else torch.float32
            )

            self.current_task = task
            print(f"✅ Successfully loaded {task} model")
            return True

        except Exception as e:
            print(f"❌ Error loading {task} model: {str(e)}")
            return False

    def record_task_feedback(self, task_name: str, rating: int, response_time: float):
        """Record user feedback for specific tasks"""
        if task_name in self.task_feedback:
            self.task_feedback[task_name]["ratings"].append(rating)
            self.task_feedback[task_name]["response_times"].append(response_time)

            # Also add to general evaluation data
            self.evaluation_data["user_satisfaction"].append(rating)
            self.evaluation_data["response_times"].append(response_time)

            # Store detailed feedback history
            feedback_entry = {
                "task": task_name,
                "rating": rating,
                "response_time": response_time,
                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            }
            self.evaluation_data["task_feedback_history"].append(feedback_entry)

    def text_summarization(self, text: str, max_length: int = 150) -> Tuple[str, float]:
        """Clean summarization output"""
        if not text.strip():
            return "Please provide text to summarize.", 0.0

        start_time = time.time()
        try:
            if not self.load_pipeline_efficiently("summarization"):
                return "Failed to load summarization model.", 0.0

            result = self.current_pipeline(
                text[:1000],
                max_length=max_length,
                min_length=30,
                do_sample=False,
                truncation=True
            )

            response_time = time.time() - start_time
            self.evaluation_data["response_times"].append(response_time)

            return result[0]['summary_text'], response_time

        except Exception as e:
            return f"Error in summarization: {str(e)}", 0.0

    def sentiment_analysis(self, text: str) -> Tuple[str, float]:
        """Clean sentiment analysis output"""
        if not text.strip():
            return "Please provide text for sentiment analysis.", 0.0

        start_time = time.time()
        try:
            if not self.load_pipeline_efficiently("sentiment"):
                return "Failed to load sentiment model.", 0.0

            result = self.current_pipeline(text[:512])
            response_time = time.time() - start_time
            self.evaluation_data["response_times"].append(response_time)

            label = result[0]['label']
            score = result[0]['score']

            sentiment_map = {
                'LABEL_0': 'Negative',
                'LABEL_1': 'Neutral',
                'LABEL_2': 'Positive',
                'NEGATIVE': 'Negative',
                'POSITIVE': 'Positive'
            }

            sentiment = sentiment_map.get(label, label)
            return f"Sentiment: {sentiment}\nConfidence: {score:.2%}", response_time

        except Exception as e:
            return f"Error in sentiment analysis: {str(e)}", 0.0

    def question_answering(self, question: str, context: str) -> Tuple[str, float]:
        """Enhanced QA with timing"""
        if not question.strip() or not context.strip():
            return "Please provide both question and context.", 0.0

        start_time = time.time()
        try:
            if not self.load_pipeline_efficiently("qa"):
                return "Failed to load QA model.", 0.0

            result = self.current_pipeline(
                question=question,
                context=context[:800]
            )

            response_time = time.time() - start_time
            self.evaluation_data["response_times"].append(response_time)
            return f"{result['answer']} (Confidence: {result['score']:.2%})", response_time

        except Exception as e:
            return f"Error in question answering: {str(e)}", 0.0

    def next_word_prediction(self, prompt: str) -> Tuple[str, float]:
        """Next word prediction - predicts only 1-3 words"""
        if not prompt.strip():
            return "Please provide a prompt for next word prediction.", 0.0

        start_time = time.time()
        try:
            if not self.load_pipeline_efficiently("next_word"):
                return "Failed to load next word prediction model.", 0.0

            result = self.current_pipeline(
                prompt[-100:],  # Use last 100 characters for context
                max_new_tokens=3,  # Only predict 1-3 words
                temperature=0.7,
                do_sample=True,
                pad_token_id=50256,
                return_full_text=False  # Only return new tokens
            )

            response_time = time.time() - start_time
            predicted_text = result[0]['generated_text']

            # Clean up the prediction
            predicted_text = predicted_text.strip()
            if not predicted_text:
                predicted_text = "[Unable to predict next word]"

            return predicted_text, response_time

        except Exception as e:
            return f"Error in next word prediction: {str(e)}", 0.0

    def story_generation(self, prompt: str, max_length: int = 150) -> Tuple[str, float]:
        """Enhanced story generation with better model"""
        if not prompt.strip():
            return "Please provide a story prompt.", 0.0

        start_time = time.time()
        try:
            if not self.load_pipeline_efficiently("text_generation"):
                return "Failed to load story generation model.", 0.0

            result = self.current_pipeline(
                prompt[:200],
                max_new_tokens=max_length,
                temperature=0.8,
                do_sample=True,
                pad_token_id=50256,
                repetition_penalty=1.1
            )

            response_time = time.time() - start_time
            generated_story = result[0]['generated_text']

            # Clean up the story
            if generated_story.startswith(prompt):
                generated_story = generated_story[len(prompt):].strip()

            return generated_story, response_time

        except Exception as e:
            return f"Error in story generation: {str(e)}", 0.0

    def chatbot_response(self, message: str, history: List) -> Tuple[str, List, float]:
        """Enhanced chatbot with GPT-2 Large"""
        if not message.strip():
            return "", history, 0.0

        start_time = time.time()
        try:
            if not self.load_pipeline_efficiently("chatbot"):
                error_response = "Failed to load chatbot model."
                history.append([message, error_response])
                return "", history, 0.0

            # Build conversation context more intelligently
            context = ""
            if history:
                # Use last 2 exchanges for context to avoid token limit
                recent_history = history[-2:]
                for user_msg, bot_msg in recent_history:
                    context += f"User: {user_msg}\nBot: {bot_msg}\n"

            # Enhanced prompt engineering
            full_prompt = f"{context}User: {message}\nBot:"

            # Generate response with better parameters
            result = self.current_pipeline(
                full_prompt,
                max_new_tokens=30,
                temperature=0.7,
                do_sample=True,
                pad_token_id=50256,
                eos_token_id=50256,
                repetition_penalty=1.2,
                top_p=0.9
            )

            response = result[0]['generated_text']
            response_time = time.time() - start_time

            # Extract bot response more intelligently
            if "Bot:" in response:
                bot_response = response.split("Bot:")[-1].strip()
                # Clean up response
                if "User:" in bot_response:
                    bot_response = bot_response.split("User:")[0].strip()
            else:
                bot_response = "I'm here to help! Could you please rephrase your question?"

            # Final cleanup
            bot_response = bot_response.replace("User:", "").replace("Bot:", "").strip()

            # Ensure reasonable length
            if len(bot_response) > 200:
                bot_response = bot_response[:200] + "..."

            if not bot_response or len(bot_response) < 3:
                bot_response = "I understand. Please tell me more about what you'd like to know!"

            history.append([message, bot_response])
            return "", history, response_time

        except Exception as e:
            # Catch any exceptions during chatbot response generation
            error_response = f"Error in chatbot response: {str(e)}"
            history.append([message, error_response])
            return "", history, 0.0

    def generate_image(self, prompt: str, steps: int = 20) -> tuple:
        """Actual image generation"""
        if not prompt.strip():
            return None, "Please provide a prompt for image generation."

        start_time = time.time()
        try:
            # Aggressive cleanup before loading image model
            if self.current_pipeline is not None:
                print("🧹 Cleaning up NLP models for image generation...")
                del self.current_pipeline
                self.current_pipeline = None
                self.current_task = None
                self.clear_gpu_memory()

            if not hasattr(self, 'image_pipe') or self.image_pipe is None:
                print("🎨 Loading image generation model...")
                self.image_pipe = StableDiffusionPipeline.from_pretrained(
                    "runwayml/stable-diffusion-v1-5",
                    torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,
                    use_safetensors=True
                )

                if torch.cuda.is_available():
                    self.image_pipe.enable_attention_slicing()
                    self.image_pipe.enable_sequential_cpu_offload()

            image = self.image_pipe(
                prompt,
                num_inference_steps=steps,
                width=512,
                height=512,
                guidance_scale=7.5
            ).images[0]

            response_time = time.time() - start_time
            # Immediate cleanup after generation
            self.clear_gpu_memory()
            return image, f"✅ Generated image: '{prompt}' in {response_time:.2f}s", response_time

        except Exception as e:
            # Emergency cleanup on error
            self.aggressive_memory_cleanup()
            return None, f"❌ Image generation error: {str(e)}", 0.0

    def get_evaluation_report(self) -> str:
        """Enhanced evaluation report with task-specific metrics"""
        try:
            if not any(self.task_feedback.values()):
                return "No evaluation data available yet. Use the tools to generate metrics."

            report = "# 📊 Comprehensive Model Evaluation Report\n\n"

            # Overall Statistics
            total_tasks = sum(len(data["ratings"]) for data in self.task_feedback.values())
            if total_tasks > 0:
                overall_satisfaction = np.mean([rating for data in self.task_feedback.values() for rating in data["ratings"]])
                overall_response_time = np.mean([time for data in self.task_feedback.values() for time in data["response_times"]])

                report += f"## 🎯 Overall Performance\n"
                report += f"- **Total Tasks Completed**: {total_tasks}\n"
                report += f"- **Overall Satisfaction**: {overall_satisfaction:.2f}/5.0 ⭐\n"
                report += f"- **Average Response Time**: {overall_response_time:.2f} seconds ⚡\n\n"

            # Task-specific performance
            report += "## 📈 Task-Specific Performance\n\n"

            for task_name, data in self.task_feedback.items():
                if data["ratings"]:
                    avg_rating = np.mean(data["ratings"])
                    avg_time = np.mean(data["response_times"])
                    task_count = len(data["ratings"])

                    # Determine performance emoji
                    if avg_rating >= 4.5:
                        emoji = "🔥"
                    elif avg_rating >= 4.0:
                        emoji = "🚀"
                    elif avg_rating >= 3.5:
                        emoji = "👍"
                    elif avg_rating >= 3.0:
                        emoji = "👌"
                    else:
                        emoji = "⚠️"

                    report += f"### {emoji} {task_name.replace('_', ' ').title()}\n"
                    report += f"- **Usage Count**: {task_count}\n"
                    report += f"- **Average Rating**: {avg_rating:.2f}/5.0\n"
                    report += f"- **Average Response Time**: {avg_time:.2f}s\n"
                    report += f"- **Performance Status**: {'Excellent' if avg_rating >= 4.5 else 'Good' if avg_rating >= 4.0 else 'Fair' if avg_rating >= 3.0 else 'Needs Improvement'}\n\n"

            # Recent feedback history
            if self.evaluation_data["task_feedback_history"]:
                report += "## 📝 Recent Activity\n\n"
                recent_feedback = self.evaluation_data["task_feedback_history"][-5:]  # Last 5 entries
                for entry in recent_feedback:
                    report += f"- **{entry['task'].replace('_', ' ').title()}**: {entry['rating']}/5 ⭐ | {entry['response_time']:.2f}s | {entry['timestamp']}\n"

            return report

        except Exception as e:
            return f"Error generating evaluation report: {str(e)}"

# Initialize the enhanced tool
nlp_tool = EvaluatedNLPTool()

def create_feedback_component():
    """Create reusable feedback component"""
    with gr.Row(visible=False) as feedback_row:
        gr.Markdown("### 📝 Rate this response:")
        rating = gr.Slider(1, 5, value=5, label="Satisfaction Rating", step=1)
        submit_feedback = gr.Button("Submit Feedback", variant="secondary", size="sm")
        feedback_status = gr.Textbox(label="", interactive=False, visible=False)

    return feedback_row, rating, submit_feedback, feedback_status

🚀 EvaluatedNLPTool initialized on device: cuda


# Gradio WebApp

In [5]:
# Create comprehensive Gradio interface
def create_enhanced_interface():
    """Create feature-rich Gradio interface"""

    with gr.Blocks(title="🤖 Enhanced NLP Tool", theme=gr.themes.Soft()) as demo:

        gr.Markdown("""
        <h1 align="center"> 🤖 Enhanced Multifunctional NLP & AI Tool</h1>

        <p align="center"><b> 🔥 Optimized for Google Colab with Smart Memory Management & User Feedback System <b></p>

        ## 🚀 Enhanced Features:
        - 📝 **Text Summarization**: Intelligent content condensation with feedback
        - 😊 **Sentiment Analysis**: Emotional tone detection with confidence scoring
        - ❓ **Question Answering**: Context-based intelligent responses
        - 🔮 **Next Word Prediction**: Predicts 1-3 next words from your input
        - 📚 **Story Generation**: Creative storytelling with enhanced GPT-2 Large
        - 💬 **Intelligent Chatbot**: Powered by GPT-2 Large for better conversations
        - 🎨 **Image Generation**: Text-to-image with aggressive memory management
        - 📊 **Advanced Analytics**: Task-specific performance tracking & user feedback
        """)

        with gr.Tabs():

            # Enhanced Summarization Tab
            with gr.TabItem("📝 Text Summarization"):
                with gr.Row():
                    with gr.Column():
                        sum_input = gr.Textbox(
                            lines=8,
                            placeholder="Enter text to summarize (optimized for up to 1000 characters)...",
                            label="Input Text",
                            max_lines=12
                        )
                        sum_length = gr.Slider(
                            50, 300, value=150,
                            label="Summary Length",
                            info="Adjust summary detail level"
                        )
                        sum_btn = gr.Button("📝 Generate Summary", variant="primary", size="lg")

                    with gr.Column():
                        sum_output = gr.Textbox(
                            lines=8,
                            label="Summary Output",
                            interactive=False,
                            placeholder="Your intelligent summary will appear here..."
                        )
                        sum_time = gr.Textbox(label="Response Time", interactive=False, visible=False)

                # Feedback component for summarization
                sum_feedback_row, sum_rating, sum_feedback_btn, sum_feedback_status = create_feedback_component()

                def summarize_with_feedback(text, length):
                    result, resp_time = nlp_tool.text_summarization(text, length)
                    return result, f"{resp_time:.2f}s", gr.Row.update(visible=True)

                def submit_sum_feedback(rating, resp_time_str):
                    try:
                        resp_time = float(resp_time_str.replace('s', ''))
                        nlp_tool.record_task_feedback("summarization", int(rating), resp_time)
                        return "✅ Thank you for your feedback!", gr.Row.update(visible=False)
                    except:
                        return "❌ Error submitting feedback", gr.Row.update(visible=False)

                # Wire up the buttons
                sum_btn.click(
                    summarize_with_feedback,
                    inputs=[sum_input, sum_length],
                    outputs=[sum_output, sum_time, sum_feedback_row]
                )
                sum_feedback_btn.click(
                    submit_sum_feedback,
                    inputs=[sum_rating, sum_time],
                    outputs=[sum_feedback_status, sum_feedback_row]
                )

                # Example inputs/hooks
                gr.Examples(
                    examples=[
                        ["Artificial intelligence has revolutionized numerous industries by automating complex processes, enhancing decision-making capabilities, and enabling unprecedented efficiency gains. Machine learning algorithms analyze vast datasets to identify patterns, predict outcomes, and optimize operations across healthcare, finance, transportation, and manufacturing sectors.", 100],
                        ["Climate change represents one of humanity's greatest challenges, with rising global temperatures causing sea level increases, extreme weather events, ecosystem disruptions, and biodiversity loss. Mitigation strategies include renewable energy adoption, carbon capture technologies, sustainable agriculture practices, and international cooperation frameworks.", 80]
                    ],
                    inputs=[sum_input, sum_length],
                    outputs=sum_output,
                    fn=nlp_tool.text_summarization
                )

            # Enhanced Sentiment Analysis Tab
            with gr.TabItem("😊 Sentiment Analysis"):
                with gr.Row():
                    with gr.Column():
                        sent_input = gr.Textbox(
                            lines=5,
                            placeholder="Enter text to analyze emotional tone...",
                            label="Input Text"
                        )
                        sent_btn = gr.Button("😊 Analyze Sentiment", variant="primary", size="lg")

                    with gr.Column():
                        sent_output = gr.Textbox(
                            lines=5,
                            label="Sentiment Analysis Result",
                            interactive=False
                        )
                        sent_time = gr.Textbox(label="Response Time", interactive=False, visible=False)

                sent_feedback_row, sent_rating, sent_feedback_btn, sent_feedback_status = create_feedback_component()

                def analyze_sentiment_with_feedback(text):
                    result, resp_time = nlp_tool.sentiment_analysis(text)
                    return result, f"{resp_time:.2f}s", gr.Row.update(visible=True)

                def submit_sent_feedback(rating, resp_time_str):
                    try:
                        resp_time = float(resp_time_str.replace('s', ''))
                        nlp_tool.record_task_feedback("sentiment", int(rating), resp_time)
                        return "✅ Thank you for your feedback!", gr.Row.update(visible=False)
                    except:
                        return "❌ Error submitting feedback", gr.Row.update(visible=False)

                sent_btn.click(analyze_sentiment_with_feedback, sent_input,
                              [sent_output, sent_time, sent_feedback_row])
                sent_feedback_btn.click(submit_sent_feedback, [sent_rating, sent_time],
                                       [sent_feedback_status, sent_feedback_row])

                gr.Examples(
                    examples=[
                        "I absolutely love this new feature! It works perfectly and exceeded my expectations.",
                        "This product is quite disappointing and doesn't meet the advertised specifications.",
                        "The service is adequate, nothing exceptional but gets the job done reasonably well.",
                        "I'm excited about the possibilities this technology brings to our workflow!"
                    ],
                    inputs=sent_input,
                    outputs=sent_output,
                    fn=nlp_tool.sentiment_analysis
                )

            # Enhanced Question Answering Tab - FIXED
            with gr.TabItem("❓ Question Answering"):
                with gr.Row():
                    with gr.Column():
                        qa_question = gr.Textbox(
                            lines=2,
                            placeholder="Enter your question...",
                            label="Question"
                        )
                        qa_context = gr.Textbox(
                            lines=6,
                            placeholder="Provide context containing the answer...",
                            label="Context"
                        )
                        qa_btn = gr.Button("❓ Get Answer", variant="primary", size="lg")

                    with gr.Column():
                        qa_output = gr.Textbox(
                            lines=8,
                            label="Answer with Confidence",
                            interactive=False
                        )
                        qa_time = gr.Textbox(label="Response Time", interactive=False, visible=False)

                qa_feedback_row, qa_rating, qa_feedback_btn, qa_feedback_status = create_feedback_component()

                def answer_question_with_feedback(question, context):
                    result, resp_time = nlp_tool.question_answering(question, context)
                    return result, f"{resp_time:.2f}s", gr.Row.update(visible=True)

                def submit_qa_feedback(rating, resp_time_str):
                    try:
                        resp_time = float(resp_time_str.replace('s', ''))
                        nlp_tool.record_task_feedback("qa", int(rating), resp_time)
                        return "✅ Thank you for your feedback!", gr.Row.update(visible=False)
                    except:
                        return "❌ Error submitting feedback", gr.Row.update(visible=False)

                qa_btn.click(answer_question_with_feedback, [qa_question, qa_context],
                            [qa_output, qa_time, qa_feedback_row])
                qa_feedback_btn.click(submit_qa_feedback, [qa_rating, qa_time],
                                     [qa_feedback_status, qa_feedback_row])

                gr.Examples(
                    examples=[
                        ["What is machine learning?", "Machine learning is a subset of artificial intelligence that enables computers to learn and improve from experience without being explicitly programmed. It uses algorithms to analyze data, identify patterns, and make predictions or decisions."],
                        ["When was the company founded?", "TechCorp was founded in 2010 by three university graduates who wanted to revolutionize cloud computing. The company started in a small garage and now employs over 5000 people worldwide."]
                    ],
                    inputs=[qa_question, qa_context],
                    outputs=qa_output,
                    fn=nlp_tool.question_answering
                )

            # Next Word Prediction Tab
            with gr.TabItem("🔮 Text Generation"):
                with gr.Row():
                    with gr.Column():
                        nw_input = gr.Textbox(
                            lines=3,
                            placeholder="Enter text and I'll predict the next 1-3 words...",
                            label="Text Prompt"
                        )
                        nw_btn = gr.Button("🔮 Predict Next Words", variant="primary", size="lg") # Added nw_btn definition
                    with gr.Column():
                        nw_output = gr.Textbox(
                            lines=3,
                            label="Predicted Next Words",
                            interactive=False
                        )
                        nw_time = gr.Textbox(label="Response Time", interactive=False, visible=False)


                nw_feedback_row, nw_rating, nw_feedback_btn, nw_feedback_status = create_feedback_component()

                def predict_next_words_with_feedback(text):
                    result, resp_time = nlp_tool.next_word_prediction(text)
                    return result, f"{resp_time:.2f}s", gr.Row.update(visible=True)

                def submit_nw_feedback(rating, resp_time_str):
                    try:
                        resp_time = float(resp_time_str.replace('s', ''))
                        nlp_tool.record_task_feedback("next_word", int(rating), resp_time)
                        return "✅ Thank you for your feedback!", gr.Row.update(visible=False)
                    except:
                        return "❌ Error submitting feedback", gr.Row.update(visible=False)

                nw_btn.click(predict_next_words_with_feedback, nw_input,
                            [nw_output, nw_time, nw_feedback_row])
                nw_feedback_btn.click(submit_nw_feedback, [nw_rating, nw_time],
                                     [nw_feedback_status, nw_feedback_row])

                gr.Examples(
                    examples=[
                        "The weather today is",
                        "Machine learning is a subset of",
                        "In the future, artificial intelligence will"
                    ],
                    inputs=nw_input,
                    outputs=nw_output,
                    fn=lambda x: nlp_tool.next_word_prediction(x)[0]
                )

            # Story Generation Tab
            with gr.TabItem("📚 Story Generation"):
                with gr.Row():
                    with gr.Column():
                        story_input = gr.Textbox(
                            lines=3,
                            placeholder="Enter a story prompt or beginning...",
                            label="Story Prompt"
                        )
                        story_length = gr.Slider(
                            50, 300, value=150,
                            label="Story Length",
                            info="Longer stories for more detail"
                        )
                        story_btn = gr.Button("📚 Generate Story", variant="primary", size="lg")

                    with gr.Column():
                        story_output = gr.Textbox(
                            lines=15,  # Increased height
                            label="Generated Story",
                            interactive=False,
                            max_lines=20,  # Enable scrolling
                            show_copy_button=True
                        )
                        story_time = gr.Textbox(label="Response Time", interactive=False, visible=False)

                story_feedback_row, story_rating, story_feedback_btn, story_feedback_status = create_feedback_component()

                def generate_story_with_feedback(prompt, length):
                    result, resp_time = nlp_tool.story_generation(prompt, length)
                    return result, f"{resp_time:.2f}s", gr.Row.update(visible=True)

                def submit_story_feedback(rating, resp_time_str):
                    try:
                        resp_time = float(resp_time_str.replace('s', ''))
                        nlp_tool.record_task_feedback("story_generation", int(rating), resp_time)
                        return "✅ Thank you for your feedback!", gr.Row.update(visible=False)
                    except:
                        return "❌ Error submitting feedback", gr.Row.update(visible=False)

                story_btn.click(generate_story_with_feedback, [story_input, story_length],
                               [story_output, story_time, story_feedback_row])
                story_feedback_btn.click(submit_story_feedback, [story_rating, story_time],
                                        [story_feedback_status, story_feedback_row])
                gr.Examples(
                    examples=[
                        ["Once upon a time in a digital realm", 150],
                        ["The last data scientist on Earth discovered", 180],
                        ["In the year 2050, AI and humans", 160]
                    ],
                    inputs=[story_input, story_length],
                    outputs=story_output,
                    fn=lambda prompt, length: nlp_tool.story_generation(prompt, length)[0]
                )

            # Enhanced Chatbot Tab
            with gr.TabItem("💬 Interactive Chatbot"):
                gr.Markdown("### Have an intelligent conversation with AI")

                chatbot = gr.Chatbot(
                    height=450,
                    label="AI Assistant Conversation",
                    show_label=True,
                    avatar_images=(None, "🤖")
                )

                with gr.Row():
                    msg = gr.Textbox(
                        placeholder="Type your message here...",
                        label="Your Message",
                        lines=2,
                        scale=4
                    )
                    send_btn = gr.Button("Send 📤", variant="primary", scale=1)

                clear_btn = gr.Button("🗑️ Clear Conversation", variant="secondary")
                chat_time = gr.State(0.0)

                def send_message_with_timing(message, history):
                    _, new_history, resp_time = nlp_tool.chatbot_response(message, history)
                    return "", new_history, resp_time

                msg.submit(send_message_with_timing, [msg, chatbot], [msg, chatbot, chat_time])
                send_btn.click(send_message_with_timing, [msg, chatbot], [msg, chatbot, chat_time])
                clear_btn.click(lambda: ([], ""), None, [chatbot, msg])

            # Image Generation Tab
            with gr.TabItem("🎨 Image Generation"):
                with gr.Row():
                    with gr.Column():
                        img_input = gr.Textbox(
                            lines=3,
                            placeholder="Describe the image you want to create...",
                            label="Image Prompt"
                        )
                        img_steps = gr.Slider(
                            10, 50, value=20,
                            label="Inference Steps",
                            info="More steps = better quality, slower generation"
                        )
                        img_btn = gr.Button("🎨 Generate Image", variant="primary", size="lg")
                        memory_btn = gr.Button("🧹 Clear Memory", variant="secondary")

                    with gr.Column():
                        img_output = gr.Image(label="Generated Image", type="pil")
                        img_status = gr.Textbox(label="Status", interactive=False)
                        img_time = gr.Textbox(label="Response Time", interactive=False, visible=False)
                img_feedback_row, img_rating, img_feedback_btn, img_feedback_status = create_feedback_component()

                gr.Examples(
                    examples=[
                        ["A futuristic cityscape with flying cars and neon lights", 20],
                        ["An abstract representation of data flowing through neural networks", 25],
                        ["A serene landscape with mountains and a crystal-clear lake", 30]
                    ],
                    inputs=[img_input, img_steps],
                    outputs=[img_output, img_status],
                )
                def generate_image_with_feedback(prompt, steps):
                    image, status, resp_time = nlp_tool.generate_image(prompt, steps)
                    return image, status, f"{resp_time:.2f}s", gr.Row.update(visible=True)

                def submit_img_feedback(rating, resp_time_str):
                    try:
                        resp_time = float(resp_time_str.replace('s', ''))
                        nlp_tool.record_task_feedback("image_generation", int(rating), resp_time)
                        return "✅ Thank you for your feedback!", gr.Row.update(visible=False)
                    except:
                        return "❌ Error submitting feedback", gr.Row.update(visible=False)

                def manual_memory_clear():
                    nlp_tool.aggressive_memory_cleanup()
                    return "🧹 Memory cleared successfully!"

                img_btn.click(generate_image_with_feedback, [img_input, img_steps],
                             [img_output, img_status, img_time, img_feedback_row])
                img_feedback_btn.click(submit_img_feedback, [img_rating, img_time],
                                      [img_feedback_status, img_feedback_row])
                memory_btn.click(manual_memory_clear, outputs=img_status)

            # Enhanced Evaluation Report Tab
            with gr.TabItem("📊 Advanced Analytics"):
                gr.Markdown("### 📈 Comprehensive Performance Analytics & User Feedback")

                eval_btn = gr.Button("📊 Generate Detailed Report", variant="primary", size="lg")
                eval_output = gr.Markdown(
                    value="Click the button above to generate your comprehensive evaluation report with task-specific metrics."
                )

                with gr.Row():
                    refresh_btn = gr.Button("🔄 Refresh Data", variant="secondary")

                eval_btn.click(nlp_tool.get_evaluation_report, outputs=eval_output)
                refresh_btn.click(nlp_tool.get_evaluation_report, outputs=eval_output)


    return demo

# Launch the enhanced application
demo = create_enhanced_interface()
demo.launch(
    share=True,
    debug=False,
    show_error=True,
    inbrowser=True
)

print("🎉 Enhanced NLP Tool with Universal Feedback System launched successfully!")
print("📱 Access your app using the shareable link above")

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://20abfa41af14ce1bb1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


🎉 Enhanced NLP Tool with Universal Feedback System launched successfully!
📱 Access your app using the shareable link above
